In [8]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [9]:
rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [14]:
def QLearning_Off_Policy(gridworld:GridWorld_v2.GridWorld_v2,TrajectoryPoolSize = 50,gamma = 0.99,trajectorySteps=-1, learning_rate=0.001, num_episodes=600)->GridWorld_v2.GridWorld_v2:
    """
    这是最基础的SARSA算法

    Parameters:
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))
    action_value = np.zeros((rows * columns, 5))
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
    # behavior_policy = 1000
    behavior_policy = np.ones((rows * columns, 5)) * 0.2
    epsilon = 0.5
    for episode in range(num_episodes):
        #清除输出，可以更好的展示策略
        time.sleep(0.2)
        clear_output(wait=True)
        print("episode",f"{episode}/{num_episodes}")
        
        

        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]
        
        initState=10
        initAction=random.randint(0,4)  

        TrajectoryPool = []
        TrajectoryPoolSize = 50
        for i in range(TrajectoryPoolSize):
            if trajectorySteps==-1:
                stop_when_reach_target = True
                # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
                # 这里改成behavior_policy
                Trajectory = gridworld.getTrajectoryScore(nowState=initState, 
                                                          action=initAction, 
                                                          policy=behavior_policy, 
                                                          steps=trajectorySteps, 
                                                          stop_when_reach_target=True)
            TrajectoryPool.append(Trajectory)
        
        for Trajectory in TrajectoryPool:
            steps = len(Trajectory) - 1
            for k in range(steps,-1,-1):
                #State，Action，Reward，NextState，NextAction
                tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
                cnt[tmpstate] += 1
                #Q.learning
                TD_error = action_value[tmpstate][tmpaction] - (tmpscore + gamma * action_value[nextState].max())
                action_value[tmpstate][tmpaction] -= learning_rate * TD_error

        # policy improvement
        policy = np.eye(5)[np.argmax(action_value,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
    
        #输出每个state的访问次数
        print(np.array(cnt).reshape(5,5))

        state_value = np.sum(policy * action_value,axis=1)
        mean_state_value = np.sum(policy * action_value,axis=1).mean()
        
        gridworld.showPolicy(policy)
        print(np.round(state_value,decimals=4).reshape(5,5))
        print("mean_state_value", mean_state_value)

    return gridworld
    

In [15]:
QLearning_Off_Policy(gridworld)

episode 207/600
[[142 125 118 124 131]
 [162 150 153 131 134]
 [314 181 126 113 101]
 [163  89  50  56  79]
 [109  99  60  69  74]]
➡️➡️➡️⬇️⬇️
⬆️⏫️⏩️➡️⬇️
⬆️⬅️⏬➡️⬇️
⬆️⏩️✅⏪⬇️
⬆️⏩️⬆️⬅️⬅️
[[6.0000e-04 1.3000e-03 3.0000e-03 6.9000e-03 1.6600e-02]
 [3.0000e-04 6.0000e-04 7.2000e-03 1.6300e-02 3.7500e-02]
 [1.0000e-04 1.0000e-04 2.4010e+00 3.4000e-02 8.1600e-02]
 [1.0000e-04 2.5005e+00 2.0791e+00 1.9178e+00 1.8220e-01]
 [0.0000e+00 1.2964e+00 1.8933e+00 9.7320e-01 4.3460e-01]]
mean_state_value 0.5553669026940116


KeyboardInterrupt: 